In [1]:
import sqlite3

conn = sqlite3.connect('resources/pubmed_filtered_articles.db')
cursor = conn.cursor()

cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
tables = cursor.fetchall()

print("Tables in the database:")
for table in tables:
    print(table[0])

table_name = 'articles'
num_rows = 5

cursor.execute(f"SELECT MIN(year), MAX(year) FROM {table_name};")
min_year, max_year = cursor.fetchone()
print(f"\nMinimum year in {table_name}: {min_year}")
print(f"Maximum year in {table_name}: {max_year}")

cursor.execute(f"SELECT COUNT(*) FROM {table_name};")
total_rows = cursor.fetchone()[0]
print(f"\nTotal rows in {table_name}: {total_rows}")

cursor.execute(f"SELECT * FROM {table_name} LIMIT {num_rows}")
rows = cursor.fetchall()

for row in rows:
    print(row)

cursor.execute(f"PRAGMA table_info('{table_name}');")
columns = cursor.fetchall()

print(f"\nColumns in '{table_name}' table:")
for column in columns:
    print(column[1])

conn.close()

Tables in the database:
articles

Minimum year in articles: 1897
Maximum year in articles: 2024

Total rows in articles: 942403
('33946206', 'Effect of a 6-Week Physical Education Intervention on Motor Competence in Pre-School Children with Developmental Coordination Disorder.', 'The objective of this research was to analyze the impact of an intervention program performed by a specialist in physical education (PE) to contribute to the development of motor competence (MC) in pre-school children with motor development problems. The sample consisted of 28 children (12 from the intervention group and 16 from the control group) aged between 4.1 and 5.9 years (mean = 4.71 ± 0.54) who were in the fifth and sixth grades of pre-school education in two schools from Lugo, Spain. The Movement Assessment Battery for Children-2 (MABC-2) was used for data collection. The data revealed that, in the pre- and post-test intervention groups, there are statistically significant differences in manual dexter

In [2]:
import sqlite3

conn = sqlite3.connect('resources/arxiv_papers.db')
cursor = conn.cursor()

cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
tables = cursor.fetchall()

print("Tables in the database:")
for table in tables:
    print(table[0])

table_name = 'papers'
num_rows = 5

cursor.execute(f"SELECT MIN(year), MAX(year) FROM {table_name};")
min_year, max_year = cursor.fetchone()
print(f"\nMinimum year in {table_name}: {min_year}")
print(f"Maximum year in {table_name}: {max_year}")


cursor.execute(f"SELECT COUNT(*) FROM {table_name};")
total_rows = cursor.fetchone()[0]
print(f"\nTotal rows in {table_name}: {total_rows}")

cursor.execute(f"SELECT * FROM {table_name} LIMIT {num_rows}")
rows = cursor.fetchall()

for row in rows:
    print(row)

conn.close()

Tables in the database:
papers

Minimum year in papers: 1991
Maximum year in papers: 2024

Total rows in papers: 29980
('2108.13580v1', 'Planning for Dexterous Ungrasping: Secure Ungrasping through Dexterous\n  Manipulation', "  This paper presents a robotic manipulation technique for dexterous\nungrasping. It refers to the capability of securely transferring a grasped\nobject from the gripper to the robot's environment, i.e. the inverse of\ngrasping or picking, through dexterous manipulation. The game of Go offers an\nexample: consider how the player would typically place an initially\npinch-grasped stone onto the board through the dexterous interaction between\nthe fingers, the stone, and the board. Likewise, dexterous ungrasping addresses\nthe necessity of changing the object's configuration relative to the gripper or\nthe environment in order to securely keep hold of the object. In particular, we\npresent a planning framework for determining a feasible minimum-cost motion\npath tha

## merging two tabels

In [3]:
import sqlite3
import os

if os.path.exists('resources/merged_database.db'):
    os.remove('resources/merged_database.db')

merged_conn = sqlite3.connect('resources/merged_database.db')
merged_cursor = merged_conn.cursor()

merged_cursor.execute('''
    CREATE TABLE articles (
        pmid TEXT PRIMARY KEY,
        title TEXT,
        abstract TEXT,
        year INTEGER,
        searchWord TEXT,
        mesh TEXT
    )
''')

merged_cursor.execute('''
    CREATE TABLE papers (
        id TEXT PRIMARY KEY,
        title TEXT,
        abstract TEXT,
        year INTEGER,
        searchWord TEXT
    )
''')

original_articles_conn = sqlite3.connect('resources/pubmed_filtered_articles.db')
original_articles_cursor = original_articles_conn.cursor()

original_articles_cursor.execute('SELECT * FROM articles')
articles_data = original_articles_cursor.fetchall()

merged_cursor.executemany('INSERT INTO articles VALUES (?, ?, ?, ?, ?, ?)', articles_data)

original_papers_conn = sqlite3.connect('resources/arxiv_papers.db')
original_papers_cursor = original_papers_conn.cursor()

original_papers_cursor.execute('SELECT * FROM papers')
papers_data = original_papers_cursor.fetchall()

merged_cursor.executemany('INSERT INTO papers VALUES (?, ?, ?, ?, ?)', papers_data)

merged_conn.commit()

merged_cursor.close()
merged_conn.close()
original_articles_cursor.close()
original_articles_conn.close()
original_papers_cursor.close()
original_papers_conn.close()

In [5]:
import sqlite3

conn = sqlite3.connect('resources/merged_database.db')
cursor = conn.cursor()

cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
tables = cursor.fetchall()

print("Tables in the database:")
for table in tables:
    print(table[0])

print("\nSchema of 'articles' table:")
cursor.execute("PRAGMA table_info(articles);")
articles_schema = cursor.fetchall()
for column in articles_schema:
    print(f"{column[1]} {column[2]}")

print("\nSchema of 'papers' table:")
cursor.execute("PRAGMA table_info(papers);")
papers_schema = cursor.fetchall()
for column in papers_schema:
    print(f"{column[1]} {column[2]}")

cursor.close()
conn.close()

data_list = []
with open('searchWord_2.txt', 'r', encoding='utf-8') as file:
        for line in file:
            data_list.append(line.strip())



conn = sqlite3.connect('resources/merged_database.db')
cursor = conn.cursor()

if os.path.exists('resources/filtered_database.db'):
    os.remove('resources/filtered_database.db')

new_conn = sqlite3.connect('resources/filtered_database.db')
new_cursor = new_conn.cursor()

new_cursor.execute('''
    CREATE TABLE articles (
        pmid TEXT PRIMARY KEY,
        title TEXT,
        abstract TEXT,
        year INTEGER,
        searchWord TEXT,
        mesh TEXT
    )
''')

new_cursor.execute('''
    CREATE TABLE papers (
        id TEXT PRIMARY KEY,
        title TEXT,
        abstract TEXT,
        year INTEGER,
        searchWord TEXT
    )
''')

for word in data_list:
    cursor.execute("SELECT * FROM articles WHERE searchWord = ?", (word,))
    rows = cursor.fetchall()
    new_cursor.executemany("INSERT INTO articles VALUES (?, ?, ?, ?, ?, ?)", rows)


for word in data_list:
    cursor.execute("SELECT * FROM papers WHERE searchWord = ?", (word,))
    rows = cursor.fetchall()
    new_cursor.executemany("INSERT INTO papers VALUES (?, ?, ?, ?, ?)", rows)


new_conn.commit()


cursor.close()
conn.close()
new_cursor.close()
new_conn.close()

Tables in the database:
articles
papers

Schema of 'articles' table:
pmid TEXT
title TEXT
abstract TEXT
year INTEGER
searchWord TEXT
mesh TEXT

Schema of 'papers' table:
id TEXT
title TEXT
abstract TEXT
year INTEGER
searchWord TEXT


In [7]:

conn = sqlite3.connect('resources/filtered_database.db')
cursor = conn.cursor()


cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
tables = cursor.fetchall()

for table_name in tables:
    table_name = table_name[0]
    print(f"tables: {table_name}")
    

    cursor.execute(f"PRAGMA table_info({table_name});")
    schema = cursor.fetchall()
    print("schema:")
    for column in schema:
        print(f"- {column[1]} {column[2]}")

    cursor.execute(f"SELECT COUNT(*) FROM {table_name};")
    count = cursor.fetchone()[0]
    print(f"Rows: {count}\n")


conn.close()

tables: articles
schema:
- pmid TEXT
- title TEXT
- abstract TEXT
- year INTEGER
- searchWord TEXT
- mesh TEXT
Rows: 577574

tables: papers
schema:
- id TEXT
- title TEXT
- abstract TEXT
- year INTEGER
- searchWord TEXT
Rows: 8586



### DB to CSV

In [1]:
import sqlite3
import csv
import os

conn = sqlite3.connect('resources/filtered_database.db')
cursor = conn.cursor()

os.makedirs('resources/output_csv', exist_ok=True)

cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
tables = cursor.fetchall()

for table_name in tables:
    table_name = table_name[0]
    print(f"Exporting {table_name} table to CSV...")

    cursor.execute(f"SELECT * FROM {table_name};")
    rows = cursor.fetchall()

    headers = [column[0] for column in cursor.description]

    csv_file_path = f"resources/output_csv/{table_name}.csv"
    with open(csv_file_path, 'w', newline='', encoding='utf-8') as csv_file:
        csv_writer = csv.writer(csv_file)
        csv_writer.writerow(headers)
        csv_writer.writerows(rows)

    print(f"{table_name}.csv file saved")

print("CSV export completed for all tables.")

conn.close()

Exporting articles table to CSV...
articles.csv file saved
Exporting papers table to CSV...
papers.csv file saved
CSV export completed for all tables.
